# Automated sunspots detection based on morphological operators
This notebook consists of an algorithm based on morphological operators to segment sunspots on H-Alpha images obtained by the spectroheliograph of Coimbra Observatory (OGAUC) and the Solar Dynamics Observatory mission (SDO).

The set available to test the algorithm consists of two pairs of images of these sensors from the same day. For that, use each of the following names as 'img_in':

2014oct23_OGAUC and 2014nov29_OGAUC; 2014oct23_SDO and 2014nov29_SDO

Some of the parameters of the algorithm may need to be tuned in each image for obtaining a better result.


## 0. Configuration 
Importing the necessary modules and defining the size of the window to display the images.

In [1]:
import diplib as dip
import matplotlib as plt
plt.rcParams["figure.figsize"] = (12,10)

PyDIPjavaio unavailable:
DLL load failed while importing PyDIPjavaio: The specified module could not be found.

libjvm not found
DIPlib -- a quantitative image analysis library
Version 3.1.0 (Sep 24 2021)
For more information see https://diplib.org


## 1. Reading and displaying the input image 
To test the algorithm with one of the 4 images mentioned above, just insert the name of the file in 'img_in': 

In [2]:
img_in = '2014oct23_OGAUC'
img = dip.ImageReadTIFF(img_in)
img.Show()

RuntimeError: Could not open the specified TIFF file
in function: TiffFile (D:\a\diplib\diplib\src\file_io\tiff_read.cpp at line number 59)

## 2. Creating the sun mask
Pre-processing to filter the white text at the top and bottom of the image using an opening with a structuring element disk of size 'se1':

In [ ]:
se1 = 20
ope = dip.Opening(img,se1)
ope.Show()

Thresholding of the opened image to obtain a binary mask of the sun (correspond to all non-zero pixels of the image):

In [ ]:
bin = dip.FixedThreshold(ope,1)
bin.Show()

Filling the holes (spots that might have the same intensity of the background) with an operator based on geodesic dilation:

In [ ]:
binFH = dip.FillHoles(bin)

Contour of the mask, obtained through the morphological gradient, superimposed to the input image:

In [ ]:
contSun = dip.MorphologicalGradientMagnitude(binFH)
img_ovr_sun = dip.Overlay(img,contSun)
img_ovr_sun.Show()

## 3. Spots Segmentation (umbra + penumbra)
Detection of the sunspots with the black tophat transform, obatined with a closing operator with a disk of size 'se2' as structuring element: 

In [ ]:
se2 = 50
BTH = dip.Closing(ope,se2) - ope
BTH.Show()

The binarization of the sunspots is obtained through a thresholding of the black top hat:

In [ ]:
BTHbin = dip.FixedThreshold(BTH,27)
BTHbin.Show()

The filtering by opening-reconstruction removes very small objects that are unlike to be sunspots:

In [ ]:
se3 = 1
fBTHbin = dip.OpeningByReconstruction(BTHbin,se3)
fBTHbin.Show()

Determination of the contours of the sunspots with a half-gradient (or morphological external gradient), superimposed to the input image:

In [ ]:
contSPOTS = dip.Dilation(fBTHbin) - fBTHbin
img_ovr_SPOTS = dip.Overlay(img,contSPOTS)
img_ovr_SPOTS.Show()

Save the ouput result in a standard TIFF format:

In [ ]:
img_out = img_in + '_spots.tif'
dip.ImageWriteTIFF(img_ovr_SPOTS,img_out)